## Data loading

Load the data from "gamezone-orders-data.xlsx" into a dataframe.


In [20]:
import pandas as pd

# Read the Excel file into a pandas DataFrame
df = pd.read_excel("gamezone-orders-data.xlsx")

# Display the first 5 rows of the DataFrame
df.head()

,USER_ID,ORDER_ID,PURCHASE_TS,SHIP_TS,PRODUCT_NAME,PRODUCT_ID,USD_PRICE,PURCHASE_PLATFORM,MARKETING_CHANNEL,ACCOUNT_CREATION_METHOD,COUNTRY_CODE
0,2c06175e,0001328c3c220830,2020-12-24 00:00:00,2020-12-13,Nintendo Switch,e682,168.00,website,affiliate,unknown,US
1,ee8e5bc2,0002af7a5c6100772,2020-10-01 00:00:00,2020-09-21,Nintendo Switch,e682,160.61,website,direct,desktop,DE
2,9eb4efe0,0002b8350e167074,2020-04-21 00:00:00,2020-02-16,Nintendo Switch,8d0d,151.20,website,direct,desktop,US
3,cac7cbaf,0006d06b98385729,2020-04-07 00:00:00,2020-04-04,Sony PlayStation 5 Bundle,54ed,1132.82,website,direct,desktop,AU
4,6b0230bc,00097279a2f46150,2020-11-24 00:00:00,2020-08-02,Nintendo Switch,8d0d,33.89,website,direct,desktop,TR


In [21]:
# Print the column names and their data types
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21864 entries, 0 to 21863
Data columns (total 11 columns):
 #   Column                   Non-Null Count  Dtype         
---  ------                   --------------  -----         
 0   USER_ID                  21864 non-null  object        
 1   ORDER_ID                 21864 non-null  object        
 2   PURCHASE_TS              21864 non-null  object        
 3   SHIP_TS                  21864 non-null  datetime64[ns]
 4   PRODUCT_NAME             21864 non-null  object        
 5   PRODUCT_ID               21864 non-null  object        
 6   USD_PRICE                21859 non-null  float64       
 7   PURCHASE_PLATFORM        21864 non-null  object        
 8   MARKETING_CHANNEL        21781 non-null  object        
 9   ACCOUNT_CREATION_METHOD  21781 non-null  object        
 10  COUNTRY_CODE             21826 non-null  object        
dtypes: datetime64[ns](1), float64(1), object(9)
memory usage: 1.8+ MB


## Data cleaning

Clean the data, including handling missing values, removing duplicates, and addressing any inconsistencies.

In [22]:
df.isnull().sum().to_frame(name='Missing Values')

,Missing Values
USER_ID,0
ORDER_ID,0
PURCHASE_TS,0
SHIP_TS,0
PRODUCT_NAME,0
PRODUCT_ID,0
USD_PRICE,5
PURCHASE_PLATFORM,0
MARKETING_CHANNEL,83
ACCOUNT_CREATION_METHOD,83


Fill missing ```MARKETING_CHANNEL```, ```ACCOUNT_CREATION_METHOD```, and ```COUNTRY_CODE``` values with 'Unknown'.

In [23]:
# Fill missing values in specified columns with 'Unknown'
df['MARKETING_CHANNEL'].fillna('Unknown', inplace=True)
df['ACCOUNT_CREATION_METHOD'].fillna('Unknown', inplace=True)
df['COUNTRY_CODE'].fillna('Unknown', inplace=True)

In [24]:

# Display the number of missing values per column after filling
display(df.isnull().sum().to_frame(name='Missing Values').to_markdown(numalign="left", stralign="left"))

'|                         | Missing Values   |\n|:------------------------|:-----------------|\n| USER_ID                 | 0                |\n| ORDER_ID                | 0                |\n| PURCHASE_TS             | 0                |\n| SHIP_TS                 | 0                |\n| PRODUCT_NAME            | 0                |\n| PRODUCT_ID              | 0                |\n| USD_PRICE               | 5                |\n| PURCHASE_PLATFORM       | 0                |\n| MARKETING_CHANNEL       | 0                |\n| ACCOUNT_CREATION_METHOD | 0                |\n| COUNTRY_CODE            | 0                |'

Handle the remaining missing values in the ```USD_PRICE``` column. Since there are only 5 missing values, dropping these rows is a reasonable approach.

In [25]:
# Drop rows with missing values in the 'USD_PRICE' column
df.dropna(subset=['USD_PRICE'], inplace=True)

# Display the number of missing values per column after dropping
display(df.isnull().sum().to_frame(name='Missing Values').to_markdown(numalign="left", stralign="left"))

'|                         | Missing Values   |\n|:------------------------|:-----------------|\n| USER_ID                 | 0                |\n| ORDER_ID                | 0                |\n| PURCHASE_TS             | 0                |\n| SHIP_TS                 | 0                |\n| PRODUCT_NAME            | 0                |\n| PRODUCT_ID              | 0                |\n| USD_PRICE               | 0                |\n| PURCHASE_PLATFORM       | 0                |\n| MARKETING_CHANNEL       | 0                |\n| ACCOUNT_CREATION_METHOD | 0                |\n| COUNTRY_CODE            | 0                |'

In [ ]:
# .to_markdown : Prints DataFrame in Markdown-friendly format.

df = pd.DataFrame(
    data={"animal_1": ["elk", "pig"], "animal_2": ["dog", "quetzal"]}
)
print(df.to_markdown())

Remove duplicate rows from the DataFrame to ensure data integrity.

In [26]:
# Remove duplicate rows
initial_rows = len(df)
df.drop_duplicates(inplace=True)
rows_after_dropping = len(df)

# Display the number of duplicate rows removed
print(f"Number of duplicate rows removed: {initial_rows - rows_after_dropping}")

# Display the number of rows after dropping duplicates
print(f"Number of rows after dropping duplicates: {rows_after_dropping}")

Number of duplicate rows removed: 35
Number of rows after dropping duplicates: 21824


Inspect columns for inconsistencies in data entry and standardize them where necessary. Focus on categorical columns like ```PURCHASE_PLATFORM```, ```MARKETING_CHANNEL```, ```ACCOUNT_CREATION_METHOD```, and ```COUNTRY_CODE```.

In [27]:
# Standardize capitalization in categorical columns
for col in ['PURCHASE_PLATFORM', 'MARKETING_CHANNEL', 'ACCOUNT_CREATION_METHOD', 'COUNTRY_CODE']:
    if df[col].dtype == 'object':
        df[col] = df[col].str.lower()

# Display unique values in standardized columns to check for inconsistencies
print("Unique values after standardization:")
for col in ['PURCHASE_PLATFORM', 'MARKETING_CHANNEL', 'ACCOUNT_CREATION_METHOD', 'COUNTRY_CODE']:
    display(f"--- {col} ---")
    display(df[col].unique())

Unique values after standardization:


'--- PURCHASE_PLATFORM ---'

array(['website', 'mobile app'], dtype=object)

'--- MARKETING_CHANNEL ---'

array(['affiliate', 'direct', 'email', 'social media', 'unknown'],
      dtype=object)

'--- ACCOUNT_CREATION_METHOD ---'

array(['unknown', 'desktop', 'mobile', 'tablet', 'tv'], dtype=object)

'--- COUNTRY_CODE ---'

array(['us', 'de', 'au', 'tr', 'ph', 'gb', 'br', 'be', 'fr', 'kr', 'es',
       'ua', 'at', 'in', 'il', 'pl', 'dk', 'ar', 'ch', 'cl', 'ro', 'vn',
       'jm', 'it', 'bb', 'jp', 'no', 'hu', 'ca', 've', 'nz', 'cy', 'bo',
       'co', 'nl', 'se', 'kw', 'za', 'si', 'ru', 'gr', 'lt', 'mx', 'th',
       'uy', 'cz', 'hk', 'ee', 'sg', 'kh', 'ie', 'iq', 'fi', 'tz', 'tw',
       'unknown', 'qa', 'pt', 'pe', 'my', 'ae', 'lv', 'eg', 'ci', 'bs',
       'is', 'cr', 'mt', 'ma', 'ht', 'sa', 'ke', 'jo', 'bg', 'bd', 'tt',
       'lk', 'ad', 'sn', 'ng', 'pf', 'pk', 'lu', 'kz', 'vi', 'sk', 'gt',
       'gh', 'tn', 'gl', 'ni', 'do', 'mv', 'ky', 'mn', 'bh', 'mc', 'lb',
       'kn', 'pr', 'hr', 'zw', 'fj', 'hn', 'pa', 'np', 'ug', 'je', 'bw',
       'ec', 'om', 'mm', 'mu', 'pg', 'al', 'by', 'eu', 'dz', 'ml', 'ps',
       'so', 'sv', 'mk', 'rs', 'im', 'vc', 'gu', 'mo', 'zm', 'ge', 'az',
       'bm', 'aw', 'sl', 'cm', 'ao', 'ba', 'mp', 'ap', 'ai', 'me', 'cn',
       'id', 'vu', 'py', 'bf', 're', 'mz', 'mh', 'md

In [28]:
df.head()

,USER_ID,ORDER_ID,PURCHASE_TS,SHIP_TS,PRODUCT_NAME,PRODUCT_ID,USD_PRICE,PURCHASE_PLATFORM,MARKETING_CHANNEL,ACCOUNT_CREATION_METHOD,COUNTRY_CODE
0,2c06175e,0001328c3c220830,2020-12-24 00:00:00,2020-12-13,Nintendo Switch,e682,168.00,website,affiliate,unknown,us
1,ee8e5bc2,0002af7a5c6100772,2020-10-01 00:00:00,2020-09-21,Nintendo Switch,e682,160.61,website,direct,desktop,de
2,9eb4efe0,0002b8350e167074,2020-04-21 00:00:00,2020-02-16,Nintendo Switch,8d0d,151.20,website,direct,desktop,us
3,cac7cbaf,0006d06b98385729,2020-04-07 00:00:00,2020-04-04,Sony PlayStation 5 Bundle,54ed,1132.82,website,direct,desktop,au
4,6b0230bc,00097279a2f46150,2020-11-24 00:00:00,2020-08-02,Nintendo Switch,8d0d,33.89,website,direct,desktop,tr


Verify that the ```USD_PRICE``` column contains valid numerical data and handle any non-numeric entries if found, potentially converting the column to a numeric type if it isn't already.

In [30]:
# Convert 'USD_PRICE' to numeric, coercing errors to NaN
df['USD_PRICE'] = pd.to_numeric(df['USD_PRICE'], errors='coerce')

# Check for any remaining non-numeric entries after coercion (should be 0 if coercion worked)
non_numeric_price = df['USD_PRICE'].isnull().sum()
print(f"Number of non-numeric entries in USD_PRICE after coercion: {non_numeric_price}")

Number of non-numeric entries in USD_PRICE after coercion: 0


In [31]:
# If there are any NaN values introduced by coercion, drop those rows (though none are expected based on previous steps)
if non_numeric_price > 0:
    df.dropna(subset=['USD_PRICE'], inplace=True)
    print(f"Dropped {non_numeric_price} rows with non-numeric USD_PRICE after coercion.")

# Display the data type of the 'USD_PRICE' column to confirm it's numeric
print(f"Data type of USD_PRICE column: {df['USD_PRICE'].dtype}")

Data type of USD_PRICE column: float64


---
## Data exploration

Perform exploratory data analysis (EDA) to understand the data distribution, identify trends, and find correlations between variables. Identify key questions the analysis should answer for stakeholders based on the data.

- Display descriptive statistics for 'USD_PRICE', 
- Analyze unique values and counts for categorical columns, 
- Extract and analyze date features from timestamp columns, and 
- Calculate the correlation matrix for relevant numerical features. 

In [33]:
# 1. Display descriptive statistics for 'USD_PRICE'
print("--- Descriptive statistics for USD_PRICE ---")
df['USD_PRICE'].describe()

--- Descriptive statistics for USD_PRICE ---


count    21824.000000
mean       281.192643
std        366.540014
min          0.000000
25%        126.000000
50%        168.000000
75%        355.110000
max       3146.880000
Name: USD_PRICE, dtype: float64

In [34]:
# 2. Calculate and display unique values and top occurrences for categorical columns
categorical_cols = ['PURCHASE_PLATFORM', 'MARKETING_CHANNEL', 'ACCOUNT_CREATION_METHOD', 'COUNTRY_CODE', 'PRODUCT_NAME']
print("\n--- Unique values and top occurrences for categorical columns ---")
for col in categorical_cols:
    print(f"\n--- {col} ---")
    unique_count = df[col].nunique()
    top_values = df[col].value_counts().head()
    print(f"Number of unique values: {unique_count}")
    top_values.to_frame(name='Count')


--- Unique values and top occurrences for categorical columns ---

--- PURCHASE_PLATFORM ---
Number of unique values: 2

--- MARKETING_CHANNEL ---
Number of unique values: 5

--- ACCOUNT_CREATION_METHOD ---
Number of unique values: 5

--- COUNTRY_CODE ---
Number of unique values: 151

--- PRODUCT_NAME ---
Number of unique values: 9


In [35]:
# 3. Explore the distribution of 'PURCHASE_TS' and 'SHIP_TS'
print("\n--- Distribution of PURCHASE_TS and SHIP_TS ---")
df['PURCHASE_TS'] = pd.to_datetime(df['PURCHASE_TS'])
df['SHIP_TS'] = pd.to_datetime(df['SHIP_TS'])

df['purchase_year'] = df['PURCHASE_TS'].dt.year
df['purchase_month'] = df['PURCHASE_TS'].dt.month
df['purchase_dayofweek'] = df['PURCHASE_TS'].dt.dayofweek # Monday=0, Sunday=6
df['purchase_hour'] = df['PURCHASE_TS'].dt.hour

df['ship_year'] = df['SHIP_TS'].dt.year
df['ship_month'] = df['SHIP_TS'].dt.month
df['ship_dayofweek'] = df['SHIP_TS'].dt.dayofweek
df['ship_hour'] = df['SHIP_TS'].dt.hour

print("\nPurchase Year Distribution:")
display(df['purchase_year'].value_counts().sort_index().to_frame(name='Count').to_markdown(numalign="left", stralign="left"))
print("\nPurchase Month Distribution:")
display(df['purchase_month'].value_counts().sort_index().to_frame(name='Count').to_markdown(numalign="left", stralign="left"))
print("\nPurchase Day of Week Distribution:")
display(df['purchase_dayofweek'].value_counts().sort_index().to_frame(name='Count').to_markdown(numalign="left", stralign="left"))
print("\nPurchase Hour Distribution:")
display(df['purchase_hour'].value_counts().sort_index().to_frame(name='Count').to_markdown(numalign="left", stralign="left"))

# We can similarly display ship date distributions if needed



--- Distribution of PURCHASE_TS and SHIP_TS ---


ParserError: String does not contain a date:    present at position 480

**There is an issue parsing the ```PURCHASE_TS``` column to datetime, likely due to some invalid entries. 
I will use errors='coerce' in pd.to_datetime to turn invalid parsing into NaT (Not a Time)**

In [38]:
# 3. Explore the distribution of 'PURCHASE_TS' and 'SHIP_TS'
print("\n--- Distribution of PURCHASE_TS and SHIP_TS ---")
# Convert with error handling
df['PURCHASE_TS'] = pd.to_datetime(df['PURCHASE_TS'], errors='coerce')
df['SHIP_TS'] = pd.to_datetime(df['SHIP_TS'], errors='coerce')

# Drop rows where timestamp conversion failed
df.dropna(subset=['PURCHASE_TS', 'SHIP_TS'], inplace=True)

df['purchase_year'] = df['PURCHASE_TS'].dt.year
df['purchase_month'] = df['PURCHASE_TS'].dt.month
df['purchase_dayofweek'] = df['PURCHASE_TS'].dt.dayofweek # Monday=0, Sunday=6
df['purchase_hour'] = df['PURCHASE_TS'].dt.hour

df['ship_year'] = df['SHIP_TS'].dt.year
df['ship_month'] = df['SHIP_TS'].dt.month
df['ship_dayofweek'] = df['SHIP_TS'].dt.dayofweek
df['ship_hour'] = df['SHIP_TS'].dt.hour

print("\nPurchase Year Distribution:")
display(df['purchase_year'].value_counts().sort_index().to_frame(name='Count').to_markdown(numalign="left", stralign="left"))
print("\nPurchase Month Distribution:")
display(df['purchase_month'].value_counts().sort_index().to_frame(name='Count').to_markdown(numalign="left", stralign="left"))
print("\nPurchase Day of Week Distribution:")
display(df['purchase_dayofweek'].value_counts().sort_index().to_frame(name='Count').to_markdown(numalign="left", stralign="left"))
print("\nPurchase Hour Distribution:")
display(df['purchase_hour'].value_counts().sort_index().to_frame(name='Count').to_markdown(numalign="left", stralign="left"))


--- Distribution of PURCHASE_TS and SHIP_TS ---

Purchase Year Distribution:


'|      | Count   |\n|:-----|:--------|\n| 2019 | 6610    |\n| 2020 | 13363   |\n| 2021 | 1846    |'


Purchase Month Distribution:


'|    | Count   |\n|:---|:--------|\n| 1  | 1785    |\n| 2  | 1825    |\n| 3  | 1509    |\n| 4  | 1723    |\n| 5  | 1727    |\n| 6  | 1613    |\n| 7  | 1692    |\n| 8  | 1796    |\n| 9  | 2176    |\n| 10 | 1549    |\n| 11 | 1955    |\n| 12 | 2469    |'


Purchase Day of Week Distribution:


'|    | Count   |\n|:---|:--------|\n| 0  | 3087    |\n| 1  | 3025    |\n| 2  | 3113    |\n| 3  | 3108    |\n| 4  | 3194    |\n| 5  | 3115    |\n| 6  | 3177    |'


Purchase Hour Distribution:


'|    | Count   |\n|:---|:--------|\n| 0  | 21813   |\n| 1  | 6       |'

In [42]:
print("\nPurchase Year Distribution:")
df['purchase_year'].value_counts().sort_index().to_frame(name='Count')


Purchase Year Distribution:


,Count
2019,6610
2020,13363
2021,1846


In [44]:
print("\nPurchase Month Distribution:")
df['purchase_month'].value_counts().sort_index().to_frame(name='Count')


Purchase Month Distribution:


,Count
1,1785
2,1825
3,1509
4,1723
5,1727
6,1613
7,1692
8,1796
9,2176
10,1549


In [45]:
print("\nPurchase Day of Week Distribution:")
df['purchase_dayofweek'].value_counts().sort_index().to_frame(name='Count')


Purchase Day of Week Distribution:


,Count
0,3087
1,3025
2,3113
3,3108
4,3194
5,3115
6,3177


In [46]:
print("\nPurchase Hour Distribution:")
df['purchase_hour'].value_counts().sort_index().to_frame(name='Count')


Purchase Hour Distribution:


,Count
0,21813
1,6


In [55]:
# 4. Calculate and display the correlation matrix
# Select relevant numerical columns
numerical_cols_for_corr = ['USD_PRICE', 'purchase_year', 'purchase_month', 'purchase_dayofweek', 'purchase_hour', 'ship_year', 'ship_month', 'ship_dayofweek', 'ship_hour']
correlation_matrix = df[numerical_cols_for_corr].corr()

print("\n--- Correlation matrix for numerical features ---")
correlation_matrix


--- Correlation matrix for numerical features ---


,USD_PRICE,purchase_year,purchase_month,purchase_dayofweek,purchase_hour,ship_year,ship_month,ship_dayofweek,ship_hour
USD_PRICE,1.000000,0.072347,0.018930,-0.000510,0.007995,0.071360,0.018683,0.002044,NaN
purchase_year,0.072347,1.000000,-0.264670,0.000666,0.006211,0.978591,-0.255367,0.000346,NaN
purchase_month,0.018930,-0.264670,1.000000,-0.007976,0.024492,-0.227992,0.921310,0.013646,NaN
purchase_dayofweek,-0.000510,0.000666,-0.007976,1.000000,-0.000202,-0.003414,-0.000959,-0.150848,NaN
purchase_hour,0.007995,0.006211,0.024492,-0.000202,1.000000,0.010851,-0.004638,-0.001321,NaN
ship_year,0.071360,0.978591,-0.227992,-0.003414,0.010851,1.000000,-0.284851,0.005612,NaN
ship_month,0.018683,-0.255367,0.921310,-0.000959,-0.004638,-0.284851,1.000000,-0.000310,NaN
ship_dayofweek,0.002044,0.000346,0.013646,-0.150848,-0.001321,0.005612,-0.000310,1.000000,NaN
ship_hour,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


### Formulating key questions for stakeholders based on the initial exploration.


1. Product Performance and Popularity:
- Which products are the most popular in terms of sales volume and revenue?
- How does the price of a product relate to its purchase frequency?

2. Customer Behavior and Trends:
- What are the peak times (year, month, day of week, hour) for purchases and shipping?
- Which purchase platforms (website vs. mobile app) are most used, and how do their sales volumes compare?
- What are the most effective marketing channels for driving sales?
- Are there any notable differences in purchase behavior based on the account creation method?
- Which countries contribute the most to sales revenue and volume?

3. Operational Efficiency:
- Is there a significant correlation between purchase time and ship time? (Based on the correlation matrix, this seems weak, but further investigation might be needed)
- Are there specific times or periods when shipping volume is highest, potentially impacting logistics?

These questions can guide further, more in-depth analysis and help stakeholders make informed business decisions.
